In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# set batch size/epoch count/image size
batch_size = 128
epochs = 10
IMG_HEIGHT = 185
IMG_WIDTH = 259

In [ ]:
# create a function to get the label for a particular car
@tf.function
def get_label(file_path):
    # convert path to list of components
    parts = tf.strings.split(file_path, os.path.sep)
    # the second part is the model_id
    model_id = parts[2]
    category = '0'
    f_label = open("D:/AutoLearn/Vehicle Data Set/data/misc/attributes.txt") # contains all of the image attributes sorted by model_id
    for line in f_label:
        values = line.split()
        if (values[0] == model_id): # the first value in each line is the model_id
            category = values[5] # classification label is at the end of each line in the 6th position
    f_label.close()
    return category

In [ ]:
# function to retrieve and return a resized image
@tf.function
def decode_img(img):
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_jpeg(img, channels=3)
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    img = tf.image.convert_image_dtype(img, tf.float32)
    # resize the image to the desired size.
    return tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])

In [ ]:
# function to process both the image and label from the file path
@tf.function
def process_path(file_path):
    label = get_label(file_path)
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

In [ ]:
def prepare_for_training(ds, cache=False, shuffle_buffer_size=1000):
    ds = ds.shuffle(buffer_size=shuffle_buffer_size)

    # Repeat forever
    ds = ds.repeat()

    ds = ds.batch(batch_size)

    # `prefetch` lets the dataset fetch batches in the background while the model
    # is training.
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return ds

In [ ]:
train_file = 'D:/AutoLearn/Vehicle Data Set/data/train_test_split/classification/train.txt'
test_file = 'D:/AutoLearn/Vehicle Data Set/data/train_test_split/classification/test.txt'
train_images_array = []
test_images_array = []

In [ ]:
# move file locations to array
with open(train_file) as my_file_train:
    count = 0;
    for line in my_file_train:
        train_images_array.append("Vehicle Data Set/data/image/" + line)
        count += 1

with open(test_file) as my_file_test:
    count = 0
    for line in my_file_test:
        test_images_array.append("Vehicle Data Set/data/image/" + line)
        count += 1

In [ ]:
total_train = len(train_images_array)
total_test = len(test_images_array)

In [ ]:
# transform into a tensorflow dataset object
train_ds = tf.data.Dataset.from_tensor_slices(train_images_array)
test_ds = tf.data.Dataset.from_tensor_slices(test_images_array)

In [ ]:
# transform into a dataset map with labels included
labeled_training_ds = train_ds.map(process_path, num_parallel_calls = tf.data.experimental.AUTOTUNE) 
labeled_testing_ds = test_ds.map(process_path, num_parallel_calls = tf.data.experimental.AUTOTUNE)

In [ ]:
#train_image_generator = ImageDataGenerator(rescale=1./255)
#validation_image_generator = ImageDataGenerator(rescale=1./255) 

###### problem lies here - cant use flow from directory ######
#train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           #directory=train_dir,
                                                           #target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           #class_mode='binary')

#val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              #directory=validation_dir,
                                                              #target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              #class_mode='binary')

In [ ]:
# Model Convolutions/pooling
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])

In [ ]:
# Compile model
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])
model.summary()

In [ ]:
#history = model.fit_generator(
#    train_data_gen,
#    steps_per_epoch=total_train // batch_size,
#    epochs=epochs,
#    validation_data=val_data_gen,
#    validation_steps=total_val // batch_size
#)

In [ ]:
training_ds = prepare_for_training(labeled_training_ds)

image_batch, label_batch = next(iter(training_ds))


model.fit(training_ds)
model.evaluate(test_ds)